## Librerias y conexión al software

In [1]:
import os
import sys
import comtypes.client
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

AttachToInstance = False

SpecifyPath = True

# Especificar la ruta de ETABS a continuación
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 19\ETABS.exe"



# crear objeto API helper
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)

if AttachToInstance:
    try:
        # obtener el objeto ETABS activo
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            # crear una instancia del objeto ETABS a partir de la ruta especificada
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            # crear una instancia del objeto ETABS a partir del último ETABS instalado
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)

    # Comenzar la aplicacion de ETABS
    myETABSObject.ApplicationStart()

# Crear un objeto SapModel
ret = SapModel = myETABSObject.SapModel

# Iniciar Modelo
ret = SapModel.InitializeNewModel()



## Creación de las grillas y unidades

Las grillas siempre trabajan en pies, así uno le coloque otras unidades

In [2]:
# VARIABLES
pisos=5
hpiso1=4
hpiso=3
numx=4
numy=5
espaciamientox=3.5
espaciamientoy=2.5

# Crear Grillas
ret = SapModel.File.NewGridOnly(pisos,   # Número de pisos
                                hpiso*3.28084,   # altura típico de piso a partir del segundo
                                hpiso1*3.28084,   # Altura del primer nivel
                                numx,    # Número de grillas en X
                                numy,     # Número de grillas en Y
                                espaciamientox*3.28084,    # Espaciamiento en X
                                espaciamientoy*3.28084)    # Espaciamiento en Y

# Ton_m_C = 12
tonf_m_C = 12
ret = SapModel.SetPresentUnits(tonf_m_C)

ret = SapModel.View.RefreshView(0, False)

## Creación de vigas

In [3]:
#ret = SapModel.FrameObj.AddByCoord(0, 0, 4, 3, 0, 4, 'ConcBm')
#ret

In [4]:
# Iterar a través de los pisos
for nivel in range(1, pisos + 1):
    # Calcular la altura del nivel actual
    if nivel == 1:
        altura_nivel = hpiso1
    else:
        altura_nivel = hpiso * (nivel - 1) + hpiso1

    # Crear vigas horizontales por tramos
    for j in range(numy):
        for i in range(numx - 1):
            x1 = i * espaciamientox
            x2 = (i + 1) * espaciamientox
            y = j * espaciamientoy
            ret = SapModel.FrameObj.AddByCoord(x1, y, altura_nivel, x2, y, altura_nivel,"", 'ConcBm', '', 'Global')

    # Crear vigas verticales por tramos
    for i in range(numx):
        for j in range(numy - 1):
            x = i * espaciamientox
            y1 = j * espaciamientoy
            y2 = (j + 1) * espaciamientoy
            ret = SapModel.FrameObj.AddByCoord(x, y1, altura_nivel, x, y2, altura_nivel,"", 'ConcBm', '', 'Global')

            
ret = SapModel.View.RefreshView(0, False)           

## Creación de columnas

In [5]:
# Creación de columnas en las intersecciones de las grillas
for i in range(1, numx + 1):
    for j in range(1, numy + 1):
        x_coord = (i - 1) * espaciamientox
        y_coord = (j - 1) * espaciamientoy
        for k in range(1, pisos + 1):
            z_coord1 = hpiso1 * (k - 1)  if k == 1 else hpiso1+hpiso * (k - 2) 
            z_coord2 = hpiso1  if k == 1 else hpiso1+hpiso * (k - 1) 
            ret = SapModel.FrameObj.AddByCoord(
                x_coord, y_coord, z_coord1, x_coord, y_coord, z_coord2,
                "", 'ConcCol', '', 'Global')
            
ret = SapModel.View.RefreshView(0, False)

## Creación de losas

In [6]:
# Crear las losas en cada piso
for piso_actual in range(1, pisos + 1):
    if piso_actual == 1:
        altura = hpiso1
    else:
        altura = hpiso *  (piso_actual - 1)+hpiso1

    for i in range(numx-1):
        for j in range(numy-1):
            x0 = i * espaciamientox
            x1 = (i + 1) * espaciamientox
            y0 = j * espaciamientoy
            y1 = (j + 1) * espaciamientoy
            x = [x0, x1, x1, x0]
            y = [y0, y0, y1, y1]
            z = [altura] * 4
            nombre_losa = "F2"
            tipo_seccion = "Slab1"

            ret = SapModel.AreaObj.AddByCoord(
                4, x, y, z, nombre_losa, tipo_seccion
            )


ret = SapModel.View.RefreshView(0, False)

## Creación de Diafragma rígido en cada nivel

In [7]:
# Obtención de todos los puntos del modelo
[numberPts, ptNames, ptX, ptY, ptZ, ptCsys] = SapModel.PointObj.GetAllPoints()

#Asignar diafragma a un punto
#ret = SapModel.PointObj.SetDiaphragm("2", 3, "D1")

numberPts


ret = SapModel.View.RefreshView(0, False)

In [8]:
# Obtener todos los puntos
[numberPts, ptNames, ptX, ptY, ptZ, ptCsys] = SapModel.PointObj.GetAllPoints()

# Crear un diccionario para agrupar los puntos por su altura (ptZ)
points_by_height = {}

for i in range(numberPts):
    height = round(ptZ[i], 2)  # Redondear la altura para evitar problemas de precisión
    if height not in points_by_height:
        points_by_height[height] = []

    points_by_height[height].append(ptNames[i])

# Asignar diafragma rígido a puntos en la misma altura (excluyendo Z=0)
for height, point_names in points_by_height.items():
    if height != 0 and len(point_names) > 1:
        for point_name in point_names:
            ret = SapModel.PointObj.SetDiaphragm(point_name, 3, "D1")
            if ret != 0:
                print(f"No se pudo asignar el diafragma rígido a {point_name}")

ret = SapModel.View.RefreshView(0, False)                

## Asignación de empotramiento en la base

In [9]:
# Asignar resitricciones al punto de la Base

#Restraint = [True, True, True, True, True, True]
#ret = SapModel.PointObj.SetRestraint(point_name, Restraint)

In [10]:
# Obtener todos los puntos
[numberPts, ptNames, ptX, ptY, ptZ, ptCsys] = SapModel.PointObj.GetAllPoints()

# Identificar y asignar empotramiento a los puntos con Z=0
for i in range(numberPts):
    if ptZ[i] == 0:
        point_name = ptNames[i]
        Restraint = [True, True, True, True, True, True]
        ret = SapModel.PointObj.SetRestraint(point_name, Restraint)

ret = SapModel.View.RefreshView(0, False)                

## Asignación de brazos rígidos

In [11]:
ret = SapModel.SelectObj.All()
ret = SapModel.FrameObj.SetEndLengthOffset("15",   # Label del elemento al que se le da brazo rígido, No valido (se aplica a todos al mismo tiempo)
                                            False,  # True para que se coloque en la distancia automáticamente
                                              0.5, # Distancia de brazo rígido inicial
                                              0.5, # Distancia de brazo rígido final
                                              0.5, # Rigidez utilizado
                                                2) # 0 = object, 1= Group, 2=SelectedObjects
#ret = SapModel.SelectObj.All(True)

ret = SapModel.FrameObj.SetEndLengthOffset("15",   # Label del elemento al que se le da brazo rígido, No valido (se aplica a todos al mismo tiempo)
                                            True,  # True para que se coloque en la distancia automáticamente
                                              0.5, # Distancia de brazo rígido inicial
                                              0.5, # Distancia de brazo rígido final
                                              0.5, # Rigidez utilizado
                                                2) # 0 = object, 1= Group, 2=SelectedObjects

ret = SapModel.SelectObj.All(True)

## Asignación de número de modos y masa sísmica

In [12]:
   #set mass source
# 
ret = SapModel.PropMaterial.SetMassSource(2,                        # 2 es carga especificada, otros 1 y 3
                                          2,                        # 2 es el número de loadpattern
                                          ["DEAD", "LIVE"],         # Cargas de masa sísmica
                                          [1, 0.25])                # % De carga para cada caso anteriormente señalado



ret = SapModel.LoadCases.ModalEigen.SetNumberModes("Modal"         # Nombre del caso modal
                                                   , pisos*3             # Número mínimo de modos
                                                   , pisos*3)            # Número máximo de modos

ret

COMError: (-2146233077, None, ("No se puede encontrar el método 'SetNumberModes' en la interfaz o tipo 'ETABSv1.cCaseModalEigen, ETABSv1, Version=1.0.0.0, Culture=neutral, PublicKeyToken=453d728ef24c6f5e'.", 'mscorlib', None, 0, None))